In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('current device : ',device)

# 초기 랜점 값
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)    

current device :  cpu


In [16]:
# parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 1

In [17]:
# MNIST dataset
mnist_train = dsets.MNIST(root='../MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=False)

mnist_test = dsets.MNIST(root='../MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

In [18]:
# data loader
data_loader = torch.utils.data.DataLoader(mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [19]:
class simplecnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1  = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out 

In [20]:
# model
model = simplecnn().to(device)

In [21]:
# loss, optimizer
critierion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
# training
total_batch = len(data_loader)
print('Start Training')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        x_train = X.to(device)
        y_train = Y.to(device)
        
        print(x_train.shape, y_train.shape)
#         print(x_train, y_train)
        optimizer.zero_grad()
        pred = model(x_train)
        print(pred.shape)
        print()
        cost = critierion(pred, y_train)
        cost.backward()
        
        optimizer.step()
        
        avg_cost += cost / total_batch
        break
    break
        
    print('Epcoh : {:4d}/{}, Cost : {:5f}'.format(epoch, training_epochs, avg_cost))
    
print('Training Finish')

Start Training
torch.Size([1, 1, 28, 28]) torch.Size([1])
torch.Size([1, 10])
Training Finish


with torch.no_grad():
    x_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.targets.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    
    print('ACCURACY : {}'.format(accuracy.item()))

class simplecnn2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1  = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        
        self.fc = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(625, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        return out 

In [12]:
model2 = simplecnn2()
print(model)

simplecnn(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)


In [13]:
value = torch.Tensor(1, 1, 28, 28)
model2(value).shape

torch.Size([1, 10])